In [21]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
import scipy.stats as stats

from sklearn.linear_model import ElasticNet
from sklearn.model_selection import train_test_split

In [22]:
df = pd.read_csv('data/data_clean.csv')
df

,age,sex,bmi,children,smoker,charges,is_southwest,is_southeast,is_northwest,is_northeast,age_cat,18_35,36_50,51_70,71_200,bmi_cat,underweight,normal,overweight,obesity
0,19,0,27.900,0,1,16884.92400,1,0,0,0,18_35,1,0,0,0,overweight,0,0,1,0
1,18,1,33.770,1,0,1725.55230,0,1,0,0,18_35,1,0,0,0,obesity,0,0,0,1
2,28,1,33.000,3,0,4449.46200,0,1,0,0,18_35,1,0,0,0,obesity,0,0,0,1
3,33,1,22.705,0,0,21984.47061,0,0,1,0,18_35,1,0,0,0,normal,0,1,0,0
4,32,1,28.880,0,0,3866.85520,0,0,1,0,18_35,1,0,0,0,overweight,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1332,50,1,30.970,3,0,10600.54830,0,0,1,0,36_50,0,1,0,0,obesity,0,0,0,1
1333,18,0,31.920,0,0,2205.98080,0,0,0,1,18_35,1,0,0,0,obesity,0,0,0,1
1334,18,0,36.850,0,0,1629.83350,0,1,0,0,18_35,1,0,0,0,obesity,0,0,0,1
1335,21,0,25.800,0,0,2007.94500,1,0,0,0,18_35,1,0,0,0,overweight,0,0,1,0


# Split des data

In [27]:
y = df['charges']
X = df.drop(columns=['charges', 'age_cat', 'bmi_cat'])

X_train, X_test, y_train, y_test = train_test_split(X, y)
print('X_train', X_train.shape)
print('X_test', X_test.shape)
print('y_train', X_train.shape)
print('y_test', X_test.shape)

X_train (1002, 17)
X_test (335, 17)
y_train (1002, 17)
y_test (335, 17)
